In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a achivemet and award  fact about {soccer_player}")

output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

In [6]:

chain.invoke({"soccer_player": "Ronaldo"})

"One of Cristiano Ronaldo's most impressive achievements is winning the prestigious FIFA Ballon d'Or award five times, tying him with Lionel Messi for the most wins in the history of the award."

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [7]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

In [8]:
chain.invoke({"soccer_player": "Ronaldo"})

'One achievement of Cristiano '

In [9]:
prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {historical_figure}.")

In [10]:
chain = prompt | model.bind(stop=["Israel"]) | StrOutputParser()

In [11]:
result = chain.invoke({"historical_figure": "Albert Einstein"})
print(result) 

Albert Einstein was offered the presidency of 


# Combine chain


In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()

In [14]:
chain.invoke("modi")

'Narendra Modi is the current Prime Minister of India, known for his strong leadership and focus on economic development.'

In [15]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

In [16]:
composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [17]:
composed_chain.invoke({"politician": "modi"})

"Opinions on Narendra Modi's impact on humanity are divided. Some argue that his policies have led to economic growth and development in India, benefiting millions of people. Others criticize his government for human rights violations, religious intolerance, and failure to address social issues such as poverty and inequality. Ultimately, whether Modi's leadership is positive for humanity depends on one's perspective and values."

In [18]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI



prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)


model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

In [19]:
chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

In [20]:
chain2.invoke({"politician": "trump", "language": "bhojpuri"})

'डोनाल्ड ट्रम्प, संयुक्त राज्य अमेरिका के पूर्व राष्ट्रपति, के देश अमेरिका उसका किस खंड में है?'

# RAG Applicaiton//


In [21]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [22]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

In [23]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits=text_splitter.split_documents(docs)

In [24]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [27]:
retriever = vectorstore.as_retriever()

In [28]:
prompt = hub.pull("rlm/rag-prompt")

c:\Users\HP\Master-Gen-AI-Complete-Course-from-Euron\master\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [29]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [30]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [31]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [32]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable. Techniques like Chain of Thought and Tree of Thoughts help models decompose hard tasks into multiple manageable steps. Task decomposition can be done using simple prompting, task-specific instructions, or human inputs.'

In [33]:
print("The End")

The End
